In [1]:
import pickle
import pathlib

import numpy as np
import pandas as pd

# Preparando os dados

In [48]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

clean_data_path = DATA_DIR / 'processed' / 'ames_clean.pkl'

with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

/Users/marcelomarchetto/Desktop/ml/Ames-MM/data


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2877 entries, 0 to 2929
Data columns (total 70 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   MS.SubClass      2877 non-null   category
 1   MS.Zoning        2877 non-null   category
 2   Lot.Frontage     2877 non-null   float64 
 3   Lot.Area         2877 non-null   float64 
 4   Lot.Shape        2877 non-null   category
 5   Land.Contour     2877 non-null   category
 6   Lot.Config       2877 non-null   category
 7   Land.Slope       2877 non-null   category
 8   Neighborhood     2877 non-null   category
 9   Bldg.Type        2877 non-null   category
 10  House.Style      2877 non-null   category
 11  Overall.Qual     2877 non-null   category
 12  Overall.Cond     2877 non-null   category
 13  Roof.Style       2877 non-null   category
 14  Mas.Vnr.Type     2877 non-null   category
 15  Mas.Vnr.Area     2877 non-null   float64 
 16  Exter.Qual       2877 non-null   category


In [4]:
model_data = data.copy()

In [5]:
categorical_columns = []
ordinal_columns = []
for col in model_data.select_dtypes('category').columns:
    if model_data[col].cat.ordered:
        ordinal_columns.append(col)
    else:
        categorical_columns.append(col)

In [6]:
ordinal_columns

['Lot.Shape',
 'Land.Slope',
 'Overall.Qual',
 'Overall.Cond',
 'Exter.Qual',
 'Exter.Cond',
 'Heating.QC',
 'Electrical',
 'Kitchen.Qual',
 'Functional',
 'Paved.Drive',
 'Fence']

In [7]:
categorical_columns

['MS.SubClass',
 'MS.Zoning',
 'Land.Contour',
 'Lot.Config',
 'Neighborhood',
 'Bldg.Type',
 'House.Style',
 'Roof.Style',
 'Mas.Vnr.Type',
 'Foundation',
 'Bsmt.Qual',
 'Bsmt.Cond',
 'Bsmt.Exposure',
 'BsmtFin.Type.1',
 'BsmtFin.Type.2',
 'Central.Air',
 'Garage.Type',
 'Garage.Finish',
 'Sale.Type',
 'Sale.Condition',
 'Condition',
 'Exterior']

In [8]:
for col in ordinal_columns:
    codes, _ = pd.factorize(data[col], sort=True)
    model_data[col] = codes

In [9]:
model_data[ordinal_columns].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2877 entries, 0 to 2929
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Lot.Shape     2877 non-null   int64
 1   Land.Slope    2877 non-null   int64
 2   Overall.Qual  2877 non-null   int64
 3   Overall.Cond  2877 non-null   int64
 4   Exter.Qual    2877 non-null   int64
 5   Exter.Cond    2877 non-null   int64
 6   Heating.QC    2877 non-null   int64
 7   Electrical    2877 non-null   int64
 8   Kitchen.Qual  2877 non-null   int64
 9   Functional    2877 non-null   int64
 10  Paved.Drive   2877 non-null   int64
 11  Fence         2877 non-null   int64
dtypes: int64(12)
memory usage: 292.2 KB


In [10]:
data['Lot.Shape'].value_counts()

Lot.Shape
Reg    1825
IR1     960
IR2      76
IR3      16
Name: count, dtype: int64

In [11]:
model_data['Lot.Shape'].value_counts()

Lot.Shape
0    1825
1     960
2      76
3      16
Name: count, dtype: int64

In [12]:
model_data['Exterior'].value_counts()

Exterior
VinylSd    1024
HdBoard     439
MetalSd     432
Wd Sdng     401
Plywood     218
CemntBd     126
BrkFace      86
WdShing      55
Stucco       42
AsbShng      41
Other        13
Name: count, dtype: int64

In [13]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
AsbShng,False,False,False,False,False
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False


In [14]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data, drop_first=True)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False
Other,False,False,False,False,False


In [15]:
model_test = model_data.copy()
model_data = pd.get_dummies(model_data, drop_first=True)

In [16]:
for cat in categorical_columns:
    dummies = []
    for col in model_data.columns:
        if col.startswith(cat + "_"):
            dummies.append(f'"{col}"')
    dummies_str = ', '.join(dummies)
    print(f'From column "{cat}" we made {dummies_str}\n')

From column "MS.SubClass" we made "MS.SubClass_30", "MS.SubClass_50", "MS.SubClass_60", "MS.SubClass_70", "MS.SubClass_80", "MS.SubClass_85", "MS.SubClass_90", "MS.SubClass_120", "MS.SubClass_160", "MS.SubClass_190", "MS.SubClass_Other"

From column "MS.Zoning" we made "MS.Zoning_RH", "MS.Zoning_RL", "MS.Zoning_RM"

From column "Land.Contour" we made "Land.Contour_HLS", "Land.Contour_Low", "Land.Contour_Lvl"

From column "Lot.Config" we made "Lot.Config_CulDSac", "Lot.Config_FR2", "Lot.Config_FR3", "Lot.Config_Inside"

From column "Neighborhood" we made "Neighborhood_BrDale", "Neighborhood_BrkSide", "Neighborhood_ClearCr", "Neighborhood_CollgCr", "Neighborhood_Crawfor", "Neighborhood_Edwards", "Neighborhood_Gilbert", "Neighborhood_IDOTRR", "Neighborhood_MeadowV", "Neighborhood_Mitchel", "Neighborhood_NAmes", "Neighborhood_NPkVill", "Neighborhood_NWAmes", "Neighborhood_NoRidge", "Neighborhood_NridgHt", "Neighborhood_OldTown", "Neighborhood_SWISU", "Neighborhood_Sawyer", "Neighborhood_Sa

In [17]:
X = model_data.drop(columns=['SalePrice']).copy()
y = model_data['SalePrice'].copy()

In [18]:
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)


Todas as Celulas até essa foram feitas pelo professor para o inicio do trabalho de entrega. A proxima celula representa um modelo extremamente simples de uma uma Regressão Linear feita como base para o trabalho.

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()

model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest)

RMSE = np.sqrt(mean_squared_error(ytest, ypred))

error_percent = 100 * (10**RMSE - 1)
print(f'Average error is {error_percent:.2f}%')

Average error is 15.11%


## Objetivos

Neste trabalho, tínhamos como objetivo realizar uma regressão linear para prever o preço de um imóvel com base em suas características. Começamos do básico para desenvolver um modelo capaz de fazer previsões com alta precisão.

Optamos por empregar três modelos de regressão linear: `Ridge`, `Lasso` e `ElasticNet`. Cada um desses modelos utiliza diferentes tipos de regularização para aprimorar a precisão das previsões. Contudo, surge a questão: qual tipo de regularização é o mais adequado para nosso conjunto de dados? Para responder a essa pergunta, utilizamos o `GridSearchCV`. Esta ferramenta avalia todas as combinações de parâmetros fornecidas e indica o modelo com melhor desempenho.

Na célula a seguir, apresentamos a função que aplica o `GridSearchCV` a cada modelo selecionado, retornando o mais eficiente dentre eles.


In [29]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
import numpy as np
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42  # Definindo uma semente aleatória

def grid_search_regression(model, param_grid, X, y):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X, y)
    best_params = grid_search.best_params_
    best_score = np.sqrt(-grid_search.best_score_)
    return grid_search.best_estimator_, best_params, best_score


# Alphas para serem testados
alphas = np.logspace(-6, 6, 13)

Abaixo vamos fazer os testes com varios parametros de regularização para cada um dos modelos e ver qual deles tem a melhor precisão.

### Usando Grid Search para encontrar os melhores parâmetros para o modelo de `Ridge`.

In [21]:
# Primeira busca em grade com uma faixa mais ampla de alphas

print(f'alphas: {alphas}')
param_grid_ridge = {'alpha': alphas}
model_ridge, best_ridge_params, best_ridge_score = grid_search_regression(Ridge(random_state=RANDOM_SEED), param_grid_ridge, Xtrain, ytrain)

# Refinamento da busca em grade com uma faixa mais estreita de alphas
alphas_fine = np.linspace(best_ridge_params['alpha']*0.5, best_ridge_params['alpha']*1.5, 20)
param_grid_ridge_fine = {'alpha': alphas_fine}
model_ridge, best_ridge_params_fine, best_ridge_score_fine = grid_search_regression(Ridge(random_state=RANDOM_SEED), param_grid_ridge_fine, Xtrain, ytrain)

print(f"Best parameters for Ridge: {best_ridge_params_fine}, Best RMSE score: {best_ridge_score_fine}")

alphas: [1.e-06 1.e-05 1.e-04 1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03
 1.e+04 1.e+05 1.e+06]
Best parameters for Ridge: {'alpha': 5.526315789473684}, Best RMSE score: 0.05756291176968963


### Usando Grid Search para encontrar os melhores parâmetros para o modelo de `Lasso`.

In [22]:
param_grid_lasso = {'alpha': alphas}
model_lasso, best_lasso_params, best_lasso_score = grid_search_regression(Lasso(random_state=RANDOM_SEED), param_grid_lasso, Xtrain, ytrain)

alphas_fine = np.linspace(best_lasso_params['alpha']*0.5, best_lasso_params['alpha']*1.5, 20)
param_grid_lasso_fine = {'alpha': alphas_fine}
model_lasso, best_lasso_params_fine, best_lasso_score_fine = grid_search_regression(Lasso(random_state=RANDOM_SEED), param_grid_lasso_fine, Xtrain, ytrain)

print(f"Best parameters for Lasso: {best_lasso_params_fine}, Best RMSE score: {best_lasso_score_fine}")

Best parameters for Lasso: {'alpha': 7.105263157894737e-05}, Best RMSE score: 0.05757642089954971


### Usando Grid Search para encontrar os melhores parâmetros para o modelo de `ElasticNet`.

In [23]:
param_grid_elastic_net = {'alpha': alphas, 'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}
model_en, best_elastic_net_params, best_elastic_net_score = grid_search_regression(ElasticNet(random_state=RANDOM_SEED), param_grid_elastic_net, Xtrain, ytrain)

alphas_fine = np.linspace(best_elastic_net_params['alpha']*0.5, best_elastic_net_params['alpha']*1.5, 20)
param_grid_elastic_net_fine = {'alpha': alphas_fine, 'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}
model_en, best_elastic_net_params_fine, best_elastic_net_score_fine = grid_search_regression(ElasticNet(random_state=RANDOM_SEED), param_grid_elastic_net_fine, Xtrain, ytrain)


print(f"Best parameters for Elastic Net: {best_elastic_net_params_fine}, Best RMSE score: {best_elastic_net_score_fine}")

Best parameters for Elastic Net: {'alpha': 0.00013947368421052633, 'l1_ratio': 0.5}, Best RMSE score: 0.057571966986127314


In [24]:
error_percent = 100 * (10**best_ridge_score - 1)
print(f'Average error (Ridge) is {error_percent:.2f}%')
error_percent = 100 * (10**best_ridge_score_fine - 1)
print(f'Average error (Ridge with fine tuning) is {error_percent:.2f}%')
print("---------------------------------------------------------------")

error_percent = 100 * (10**best_lasso_score - 1)
print(f'Average error (Lasso) is {error_percent:.2f}%')
error_percent = 100 * (10**best_lasso_score_fine - 1)
print(f'Average error (Lasso with fine tuning) is {error_percent:.2f}%')
print("---------------------------------------------------------------")

error_percent = 100 * (10**best_elastic_net_score - 1)
print(f'Average error (Elastic Net) is {error_percent:.2f}%')
error_percent = 100 * (10**best_elastic_net_score_fine - 1)
print(f'Average error (Elastic Net with fine tuning) is {error_percent:.2f}%')

Average error (Ridge) is 14.19%
Average error (Ridge with fine tuning) is 14.17%
---------------------------------------------------------------
Average error (Lasso) is 14.19%
Average error (Lasso with fine tuning) is 14.18%
---------------------------------------------------------------
Average error (Elastic Net) is 14.18%
Average error (Elastic Net with fine tuning) is 14.18%


Podemos perceber que o resultado esta muito proximo um do outro, portanto vamos testar os modelos com os melhores parametros no conjunto inteiro de dados.

### Testando os modelos com os parâmetros encontrados pelo GridSearchCV no conjunto de teste.

In [25]:
ypred_ridge = model_ridge.predict(Xtest)
ypred_lasso = model_lasso.predict(Xtest)
ypred_en = model_en.predict(Xtest)

RMSE_ridge = np.sqrt(mean_squared_error(ytest, ypred_ridge))
RMSE_lasso = np.sqrt(mean_squared_error(ytest, ypred_lasso))    
RMSE_en = np.sqrt(mean_squared_error(ytest, ypred_en))

error_percent_ridge = 100 * (10**RMSE_ridge - 1)
error_percent_lasso = 100 * (10**RMSE_lasso - 1)
error_percent_en = 100 * (10**RMSE_en - 1)

print(f'Average error (Ridge) is {error_percent_ridge:.2f}%')
print(f'Average error (Lasso) is {error_percent_lasso:.2f}%')
print(f'Average error (Elastic Net) is {error_percent_en:.2f}%')

Average error (Ridge) is 15.21%
Average error (Lasso) is 15.31%
Average error (Elastic Net) is 15.31%


## Conclusão dos resultados

Após conseguirmos os melhores parametros e testar com o cv, decidimos testar com o conjunto inteiro de teste para ver qual modelo teve a melhor precisão, conforme ja dito anteriormente.

Podemos perceber que não tem muita diferença entre os modelos, mas o modelo de `Ridge` foi o que teve a melhor precisão, mas podemos realmente dizer isso? Podemos ver que o resultado é melhor, mas por muito pouco, por isso precisamos ver se eles tem uma diferença estatisticamente significativa.

Para vermos se existe uma diferença estatisticamente significativa entre os modelos vamos usar o teste de hipotese de `Wilcoxon` que é um teste não parametrico que testa se existe uma diferença estatisticamente significativa entre dois modelos.

# Comparando os resultados de cada modelo no conjunto de teste

In [26]:
# make a ttest to see if the difference between the two models is significant
from scipy.stats import ttest_rel

ypred_ridge = model_ridge.predict(Xtest)
ypred_lasso = model_lasso.predict(Xtest)

pvalue = ttest_rel(ypred_ridge, ypred_lasso).pvalue

if pvalue < 0.05:
    print(f'A diferença entre os modelos Ridge e Lasso é significativa, pvalue: {pvalue*100:.2f}%')
else:
    print(f'A diferença entre os modelos Ridge e Lasso não é significativa, pvalue: {pvalue*100:.2f}%')

A diferença entre os modelos Ridge e Lasso não é significativa, pvalue: 43.89%


In [27]:

# make a ttest to see if the difference between the two models is significant
from scipy.stats import ttest_rel

ypred_ridge = model_ridge.predict(Xtest)
ypred_lasso = model_en.predict(Xtest)

pvalue = ttest_rel(ypred_ridge, ypred_lasso).pvalue

if pvalue < 0.05:
    print(f'A diferença entre os modelos Ridge e Elastic net é significativa, pvalue: {pvalue*100:.2f}%')
else:
    print(f'A diferença entre os modelos Ridge e Elastic net não é significativa, pvalue: {pvalue*100:.2f}%')

A diferença entre os modelos Ridge e Elastic net não é significativa, pvalue: 41.98%


In [28]:

# make a ttest to see if the difference between the two models is significant
from scipy.stats import ttest_rel

ypred_ridge = model_lasso.predict(Xtest)
ypred_lasso = model_en.predict(Xtest)

pvalue = ttest_rel(ypred_ridge, ypred_lasso).pvalue

if pvalue < 0.05:
    print(f'A diferença entre os modelos Lasso e Elastic net é significativa, pvalue: {pvalue*100:.2f}%')
else:
    print(f'A diferença entre os modelos Lasso e Elastic net não é significativa, pvalue: {pvalue*100:.2f}%')

A diferença entre os modelos Lasso e Elastic net não é significativa, pvalue: 45.93%


## Conclusão da Análise de Modelos

Nesta análise, exploramos três modelos de regressão com regularização: `Ridge`, `Lasso` e `ElasticNet`. Cada modelo foi cuidadosamente ajustado e validado usando uma busca em grade para determinar os hiperparâmetros que minimizam o erro de validação cruzada. A performance de cada modelo foi avaliada usando o Root Mean Square Error (RMSE), permitindo uma comparação objetiva baseada na precisão das previsões.

Observou-se que o modelo `Ridge` apresentou um desempenho ligeiramente melhor em comparação com os modelos `Lasso` e `ElasticNet`, com base no erro percentual médio no conjunto de teste. No entanto, é essencial notar que as diferenças no desempenho entre os modelos `não foram estatisticamente significativas`, conforme evidenciado pelos testes t pareados. Isso sugere que, apesar das diferenças observadas, não há uma confirmação estatística robusta de que o modelo `Ridge` seja superior.

Além disso, é importante considerar que apenas um subconjunto de modelos de regressão foi explorado nesta análise. A escolha de usar `Ridge`, `Lasso` e `ElasticNet` foi fundamentada e lógica, dado o contexto e os dados, mas existem muitos outros modelos e técnicas que poderiam ser considerados.

As consequências do desempenho do modelo final para a aplicação de negócios são significativas. Um modelo de regressão bem ajustado pode oferecer insights valiosos, facilitar a tomada de decisões e otimizar processos de negócios. No entanto, a falta de diferença estatisticamente significativa entre os modelos testados sugere que a escolha do modelo pode não impactar significativamente os resultados práticos na aplicação de negócios. Isso implica que outros fatores, como a interpretabilidade do modelo e a facilidade de implementação, podem ser considerados prioritários na seleção do modelo para uso prático.

Em conclusão, enquanto o modelo `Ridge` mostrou um desempenho marginalmente melhor nesta análise, a diferença não foi estatisticamente significativa. Futuras melhorias podem incluir a exploração de modelos de regressão alternativos, ou uma reavaliação da análise de dados e 'feature engineering' para refinar o modelo. `(Importante ressaltar que usamos um modelo de dados pré-processados já fornecidos pelo professor, não fizemos quaisquer alterações nos dados)`.

- Texto com ajuda de `Chat-GPT`